In [ ]:
import os
import sys
import multiprocessing as mp

# a hack to import module from different directory
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import tqdm
import numpy as np
import commpy as cp
from commpy.modulation import QAMModem
from commpy.channelcoding import Trellis

from radioml.radio_receivers import ModularReceiver
from radioml.decoders import ViterbiDecoder, NeuralDecoder
from radioml.demodulators import ClassicDemodulator, NeuralDemodulator
import radioml.utils as utils

## Define Paramters for the comparison

In [ ]:
class Params:
    SNR_RANGE = np.linspace(0.0, 20, 5)
    NUM_SAMPLES = 10
    BLOCK_LENGTH = 100
    BPSK, QPSK, QAM16, QAM64 = 2, 4, 16, 64
    
# For encoding/decoding convolutional codes
G = np.array([[0o7, 0o5]]) 
M = np.array([2])
trellis = Trellis(M, G, feedback=0o7, code_type='rsc')
modem = QAMModem(m=Params.QPSK)

## Define Baseline (Class Demod + Viterbi) & Modular Receivers (Class Demod + RNN)

In [ ]:
# Define a Baseline Receiver
classic_demod   = ClassicDemodulator(modem)
viterbi_decoder = ViterbiDecoder(trellis, tb_depth=15, decoding_type='hard')
neural_decoder  = NeuralDecoder(model_path='../models/BiGRU_BSC.hdf5', block_length=100)


baseline_radio = ModularReceiver(classic_demod, viterbi_decoder)
neural_radio  =  ModularReceiver(classic_demod, neural_decoder)

## Evaluate Baseline  on multiple SNRs

* For each SNR, generate (orignal message bits, modulated signals, noisy signals) using `SignalGenerator`.
* Run demodulation of baseline and modular version (a NN).
* Run decoder of baseline (viterbi) and modular version (a RNN).
* Log and results and plot

In [ ]:
def generate_signal_over_awgn(block_length=100, snr_dB=15.0):
    message_bits   = np.random.randint(0, 2, block_length)
    encoded_bits   = cp.channelcoding.conv_encode(message_bits, trellis)
    modulated_bits = modem.modulate(encoded_bits)
    corrupted_bits = cp.channels.awgn(modulated_bits, snr_dB, rate=1/2)
    
    return message_bits, modulated_bits, corrupted_bits

def signal_generator(num_examples, block_length, snr_dB):
    with mp.Pool(mp.cpu_count()) as pool:
        result = pool.starmap(generate_signal_over_awgn,
                iterable=[(block_length, snr_dB) \
                          for i in range(num_examples)])
        orignal_msg_bits, moded_bits, noisy_outputs = zip(*result)
    return (np.array(orignal_msg_bits), np.array(moded_bits), np.array(noisy_outputs))

In [ ]:

ber_logs, bler_logs = [], []
for i, snr in tqdm.tqdm(enumerate(Params.SNR_RANGE), total=len(Params.SNR_RANGE)):
    (original_bits, clean_signals, noisy_signals) = signal_generator(
        Params.NUM_SAMPLES, 
        Params.BLOCK_LENGTH, 
        snr)
    
    # ####################################
    # Run Baseline/Neral Receiver
    # ####################################
    with mp.Pool(mp.cpu_count()) as pool:
        baseline_estimated = pool.map(baseline_radio, [i for i in noisy_signals])
  
    nn_estimated = neural_radio(noisy_signals, True)
    # ####################################
    # Measure Accuracy / error
    # #################################### 
    # For decoder
    ber, bler        = utils.get_ber_bler(np.array(baseline_estimated)[:, :100], original_bits)
    nn_ber, nn_bler  = utils.get_ber_bler(nn_estimated, original_bits)
    
    ber_logs.append([ber, nn_ber])
    bler_logs.append([bler, nn_bler])
    
    print('SNR_dB = %f' % snr)
    print('\t[Baseline] Ber = {:.4f} | Bler ={:.4f} '.format(ber, bler))
    print('\t[Modular]  Ber = {:.4f} | Bler ={:.4f}'.format(nn_ber, nn_bler))

### Compare Decoder Performance 

In [ ]:
_, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
utils.visualize_ber_bler(ax1, ax2, ber_logs, bler_logs, Params.SNR_RANGE)